In [1]:
import pandas as pd
import numpy as np

In [160]:
# Create the frames
Highschooler = pd.DataFrame({
        'ID':[1510,1689,1381,1709, 1101, 1782, 1468,
              1641,1247, 1316,1911, 1501,1304,1025,
              1934,1661],
        'name':['Jordan', 'Gabriel', 'Tiffany', 
                'Cassandra','Haley', 'Andrew',
                'Kris', 'Brittany', 'Alexis',
                'Austin', 'Gabriel', 'Jessica',
                'Jordan', 'John', 'Kyle', 'Logan'],
        'grade':[9, 9, 9, 9, 10, 10, 10, 10, 11,
                 11, 11, 11, 12, 12, 12, 12]
    })
Friend = pd.DataFrame({
        'ID1':[1510, 1510, 1689, 1381, 1709, 1689,
               1782, 1782, 1782, 1468, 1468, 1101,
               1247, 1247, 1911, 1501, 1316, 1934,
               1304, 1661, 1381, 1689, 1709, 1247, 
               1247, 1782, 1468, 1316, 1304, 1101, 
               1641, 1641, 1911, 1501, 1501, 1934, 
               1934, 1304, 1661, 1025],
        'ID2':[1381, 1689, 1709, 1247, 1247, 1782,
               1468, 1316, 1304, 1101, 1641, 1641,
               1911, 1501, 1501, 1934, 1934, 1304,
               1661, 1025, 1510, 1510, 1689, 1381, 
               1709, 1689, 1782, 1782, 1782, 1468, 
               1468, 1101, 1247, 1247, 1911, 1501, 
               1316, 1934, 1304, 1661, ]
    })
Likes = pd.DataFrame({
        'ID1':[1689, 1709, 1782, 1911, 1247, 1641, 
               1316, 1501, 1934, 1025],
        'ID2':[1709, 1689, 1709, 1247, 1468, 1468, 
               1304, 1934, 1501, 1101]
    })

In [132]:
# Find the names of all students who are friends with someone
# named Gabriel. 
gabriels = Highschooler[Highschooler['name']=='Gabriel'][['ID']]
gabriels_friends = gabriels.merge(Friend, left_on= 'ID', right_on='ID1')[['ID2']]\
                           .merge(Highschooler[['ID','name']], left_on='ID2', right_on='ID')\
                           .drop_duplicates()[['name']]
gabriels_friends

,name
0,Cassandra
1,Andrew
2,Jordan
3,Jessica
4,Alexis


In [133]:
# For every student who likes someone 2 or more grades
# younger than themselves, return that student's name 
# and grade, and the name and grade of the student they
# like. 
likes_grades = Likes.merge(Highschooler, left_on='ID1', right_on='ID')\
                    .drop('ID', axis=1)\
                    .merge(Highschooler, left_on='ID2', right_on='ID',
                           suffixes=('_likes', ''))\
                    .drop('ID', axis=1)
likes_younger_grades = likes_grades[(likes_grades['grade_likes']-likes_grades['grade']) >=2]
out_cols = ['name_likes', 'name', 'grade_likes', 'grade']
likes_younger_grades[out_cols]

,name_likes,name,grade_likes,grade
9,John,Haley,12,10


In [134]:
# For every pair of students who both like each other, return 
# the name and grade of both students. Include each pair only
# once, with the two names in alphabetical order. 
mutal_like = Likes.merge(Likes, left_on='ID2', right_on='ID1', suffixes=('_first', '_second'))
mutal_like = mutal_like[mutal_like['ID1_first'] == mutal_like['ID2_second']][['ID1_first',
                                                                              'ID2_first']]
mutal_like = mutal_like[mutal_like['ID1_first'] > mutal_like['ID2_first']]
mutal_like.merge(Highschooler, left_on='ID1_first', right_on='ID')\
          .drop(['ID1_first','ID'], axis=1)\
          .merge(Highschooler, left_on='ID2_first', right_on='ID', suffixes=('_first', '_second'))\
          .drop(['ID2_first','ID'], axis=1)


,grade_first,name_first,grade_second,name_second
0,9,Cassandra,9,Gabriel
1,12,Kyle,11,Jessica


In [135]:
# Find all students who do not appear in the Likes table 
# (as a student who likes or is liked) and return their 
# names and grades. Sort by grade, then by name within 
# each grade. 
doesnt_like = Highschooler.merge(Likes[['ID1']], how='left', left_on='ID', right_on='ID1')
doesnt_like = doesnt_like[doesnt_like['ID1'].isnull()]
doesnt_like = doesnt_like.merge(Likes[['ID2']], how='left', left_on='ID', right_on='ID2')
doesnt_like = doesnt_like[doesnt_like['ID2'].isnull()]
doesnt_like[['name','grade']].sort_values(by=['grade', 'name'])

,name,grade
0,Jordan,9
1,Tiffany,9
6,Logan,12


In [136]:
# For every situation where student A likes student B, 
# but we have no information about whom B likes 
# (that is, B does not appear as an ID1 in the Likes table),
# return A and B's names and grades
like_like = Likes.merge(Likes, how='left', left_on='ID2', right_on='ID1', suffixes=('_first', '_second'))
like_like = like_like.loc[like_like['ID1_second'].isnull(), ['ID1_first','ID2_first']]
like_like.merge(Highschooler, left_on='ID1_first', right_on='ID')\
         .drop(['ID', 'ID1_first'], axis=1)\
         .merge(Highschooler, left_on='ID2_first', right_on='ID', suffixes=('_first', '_second'))\
         .drop(['ID', 'ID2_first'], axis=1)

,grade_first,name_first,grade_second,name_second
0,11,Alexis,10,Kris
1,10,Brittany,10,Kris
2,11,Austin,12,Jordan
3,12,John,10,Haley


In [148]:
# Find names and grades of students who only have friends
# in the same grade. Return the result sorted by grade, 
# then by name within each grade. 
friend_info = Friend.merge(Highschooler, left_on= 'ID1', right_on='ID')\
                    .drop('ID', axis=1)\
                    .merge(Highschooler, left_on='ID2', right_on='ID', suffixes=('_f1', '_f2'))\
                    .drop('ID', axis=1)


out_cols = ['grade_f1', 'name_f1']
def is_same_grade(g):
    one_grade = g['grade_f2'].unique().size == 1
    return (one_grade) and (g['grade_f1'].values[0] == g['grade_f2'].values[0])
friend_info.groupby('ID1')\
           .filter(is_same_grade)\
           .drop_duplicates(subset='ID1')[out_cols]\
           .sort_values(by=['name_f1','grade_f1'])

,grade_f1,name_f1
19,10,Brittany
15,11,Gabriel
18,10,Haley
38,12,John
0,9,Jordan
8,10,Kris
24,12,Logan


In [149]:
# For each student A who likes a student B where the two
# are not friends, find if they have a friend C in common
# (who can introduce them!). For all such trios, return the
# name and grade of A, B, and C. 
friend_like = Likes.merge(Friend, on='ID1', suffixes=('_x', '_y'))\
                   .rename(columns=({'ID2_x':'ID1_likes', 'ID2_y':'ID1_friends_with'}))
def not_friends(g):
    not_friends = sum(g['ID1_friends_with'] != g['ID1_likes'])
    return not_friends==len(g)
friend_like = friend_like.groupby('ID1')\
                         .filter(not_friends)
friend_like = friend_like.merge(Friend, left_on='ID1_likes', right_on='ID1', suffixes=('','_x'))\
                         .drop('ID1_x', axis=1)\
                         .rename(columns=({'ID2':'ID2_friends_with'}))
friend_like = friend_like[friend_like['ID1_friends_with']==friend_like['ID2_friends_with']]
friend_like
friend_like = friend_like.merge(Highschooler, left_on='ID1', right_on='ID')\
                         .drop('ID', axis=1)\
                         .merge(Highschooler, left_on='ID1_likes', right_on='ID', 
                                suffixes=('','_ID1_likes'))\
                         .drop('ID', axis=1)\
                         .merge(Highschooler, left_on='ID1_friends_with', right_on='ID', 
                                suffixes=('', '_ID1_friends_with'))
out_cols = ['name', 'grade', 'name_ID1_likes', 'grade_ID1_likes', 
            'name_ID1_friends_with', 'grade_ID1_friends_with']
friend_like[out_cols]

,name,grade,name_ID1_likes,grade_ID1_likes,name_ID1_friends_with,grade_ID1_friends_with
0,Andrew,10,Cassandra,9,Gabriel,9
1,Austin,11,Jordan,12,Kyle,12
2,Austin,11,Jordan,12,Andrew,10


In [151]:
# Find the difference between the number of students in the 
# school and the number of different first names. 
number_students = Highschooler['ID'].notnull().sum()
number_of_names = Highschooler['name'].unique().size
number_students - number_of_names

2

In [162]:
# Find the name and grade of all students who are liked by
# more than one other student. 
def is_popular(g):
    return g.loc[g['ID1'].notnull(),'ID1'].unique().size > 1
popular = Likes.groupby('ID2')\
               .filter(is_popular)[['ID2']]\
               .drop_duplicates()
out_cols = ['name', 'grade']
popular.merge(Highschooler, left_on='ID2', right_on='ID')[out_cols]

,name,grade
0,Cassandra,9
1,Kris,10


In [172]:
# For every situation where student A likes student B, but
# student B likes a different student C, return the names
# and grades of A, B, and C.
like_like = Likes.merge(Likes, left_on='ID2', right_on='ID1', suffixes=('', '_likes'))\
                 .drop('ID1_likes', axis=1)

def no_mutal_like(g):
    return (sum(g['ID1'] != g['ID2_likes']) == len(g))
like_like = like_like.groupby(['ID1','ID2']).filter(no_mutal_like)
like_like.merge(Highschooler, left_on='ID1', right_on='ID')\
         .drop('ID', axis=1)\
         .merge(Highschooler, left_on='ID2', right_on='ID', suffixes=('','_ID2'))\
         .drop('ID', axis=1)\
         .merge(Highschooler, left_on='ID2_likes', right_on='ID', suffixes=('', '_ID2_likes'))\
         .drop(['ID','ID1','ID2','ID2_likes'], axis=1)

,grade,name,grade_ID2,name_ID2,grade_ID2_likes,name_ID2_likes
0,10,Andrew,9,Cassandra,9,Gabriel
1,11,Gabriel,11,Alexis,10,Kris


In [178]:
# Find those students for whom all of their friends are in 
# different grades from themselves. Return the students' 
# names and grades. 
friend_info = Friend.merge(Highschooler, left_on='ID1', right_on='ID')\
                    .drop('ID', axis=1)\
                    .merge(Highschooler, left_on='ID2', right_on='ID', suffixes=('', '_friend'))
out_cols = ['name', 'grade']
def is_grade_different(g):
    return sum(g['grade'] != g['grade_friend']) == len(g)
friend_info.groupby('ID1')\
           .filter(is_grade_different)\
           .drop_duplicates(subset='ID1')[out_cols]

,name,grade
9,Austin,11


In [181]:
# What is the average number of friends per student? 
# (Your result should be just one number.) 
Friend.groupby('ID1').agg({'ID2':'count'})['ID2'].mean()

2.5

In [201]:
# Find the number of students who are either friends with 
# Cassandra or are friends of friends of Cassandra. Do not 
# count Cassandra, even though technically she is a friend
# of a friend. 
cassandra_id = Highschooler.loc[Highschooler['name']=='Cassandra', 'ID'].values[0]
cassandra_friends = Friend[Friend['ID1']==cassandra_id]
friend_friend = Friend.merge(Friend, left_on='ID2', right_on='ID1', suffixes=('','_friend'))\
                      .drop('ID1_friend', axis=1)
is_friend_friend = (friend_friend['ID1'] == cassandra_id) & (friend_friend['ID2_friend'] != cassandra_id)
friend_friend = friend_friend[is_friend_friend]
friend_friend = friend_friend.drop('ID2', axis=1)\
                             .rename(columns=({'ID2_friend':'ID2'}))\
                             .drop_duplicates()
pd.concat([cassandra_friends, friend_friend]).drop_duplicates().size


14

In [209]:
# Find the name and grade of the student(s) with the
# greatest number of friends. 
friend_count = Friend.groupby('ID1').agg({'ID2':'count'})\
                     .reset_index()\
                     .rename(columns=({'ID2':'friend_count'}))
max_friends = friend_count['friend_count'].max()
popular_kids = friend_count.loc[friend_count['friend_count'] == max_friends, ['ID1']]
popular_kids.rename(columns=({'ID1':'ID'}))
popular_kids.merge(Highschooler)\
            .drop('ID',axis=1)

MergeError: No common columns to perform merge on